## Get Transmon Parameters

In [11]:
import numpy as np
import scqubits as scq

def tmon_model(EJ, EC):

    tmon = scq.Transmon(
        EJ=EJ,
        EC=EC,
        ng=0.0,
        ncut=31
    )
    
    tmon.eigenvals(evals_count=12)

    freq = np.diff(tmon.eigenvals(evals_count=12))[0]
    alpha = np.diff(np.diff(tmon.eigenvals(evals_count=12)))[0]

    return freq, alpha

def cost(x, *args):
    
    EJ = x[0]
    EC = x[1]
    
    freq_exp = args[0][0]
    alpha_exp = args[0][1]

    
    freq_model, alpha_model = tmon_model(EJ, EC)
    
    exp = np.array([freq_exp, alpha_exp])
    model = np.array([freq_model, alpha_model])
    
    error = np.sum( ((model-exp) / exp)**2 )
    
    return error

import scipy.optimize

def get_transmon_hamiltonian(freq, alpha):

    x0 = np.array([4, 1])

    args = [freq, alpha]

    result = scipy.optimize.minimize(cost, x0, tol=1e-3,bounds=[(0,100),(0,100)],args=args)

    result['x']
    
    print(result)

    tmon = scq.Transmon(
            EJ=9.44645349,
            EC=0.19360087,
            ng=0.0,
            ncut=31)
    
    return tmon.hamiltonian()

H = get_transmon_hamiltonian(4.66e9, 160e6)
# w_q = np.diff(np.sort(np.linalg.eig(H)[0]))[0]  # bare qubit frequency
# alpha = np.diff(np.diff(np.sort(np.linalg.eig(H)[0])))[0]  # anharmonicity
# k6 = np.diff(np.diff(np.diff(np.sort(np.linalg.eig(H)[0]))))[0]  # sixth order term
# print(w_q)
# print(alpha)
# print(k6)
# np.diff(np.sort(np.linalg.eig(H)[0]))

  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: 2.0000000438461325
        x: [ 4.000e+00  1.000e+00]
      nit: 0
      jac: [ 0.000e+00  8.882e-08]
     nfev: 3
     njev: 1
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>


In [7]:
print(tmon_model(4, 1))

(5.186189591553678, -3.685757164692565)


Verify that the extracted transmon parameters give the same list of energies as the scqubits model.

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from qutip import *


Nq = 10
w_q  = 2 * np.pi * 3.62e9 # bare qubit frequency
alpha = -223e6 * 2 * np.pi # anharmonicity
k6 = -26e6 * 2 * np.pi # sixth order term

q = destroy(Nq)

tlist = np.linspace(0,1e-7,10000)

H0 = w_q * q.dag() * q + (alpha/2)*q.dag()*q.dag()*q*q + (k6/6)*(q.dag()*q.dag()*q.dag()*q*q*q)

np.diff(H0.eigenenergies()/(2e9*np.pi))


array([3.62 , 3.397, 3.148, 2.873, 2.572, 2.245, 1.892, 1.513, 1.108])

Create a dispersive readout Hamiltonian and use the eigenvectors to pick out which energies correspond to which cavity and transmon photon number. Then find the difference in energies between zero and one cavity photon, for all values of transmon photon number. This should give a state dependent $\chi$, keeping in mind that the transmon Hamiltonian is only up to sixth order. We could fit everything, but here I'm just playing with values to give close numbers.

The data we have unicthe dressed cavity frequency for four observed qubit states, and the dressed qubit transition frequency for the first and second transitions (equivalent to the anharmonicity).

From this we can get the fit bare cavity frequency.

The cavity frequencies are 5.72965, 5.72931, 5.72896 and 5.72867 GHz. The odd extra peaks are at (roughly) 5.7267 and 5.7305 GHz. 

The qubit frequencies are 3.620 GHz and 3.397 GHz. 

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from qutip import *

Nc = 6
Nq = 6

w_c  = 2 * np.pi * 5.7283e9 # bare resonator frequency
w_q  = 2 * np.pi * 3.623e9 # bare qubit frequency
alpha = -223e6 * 2 * np.pi # anharmonicity
g = 59.5e6 * 2 * np.pi
k6 = -26e6 * 2 * np.pi # sixth order term

c  = tensor(destroy(Nc), qeye(Nq))

q = tensor(qeye(Nc), destroy(Nq))

tlist = np.linspace(0,1e-7,10000)

H0 = w_c * c.dag() * c + w_q * q.dag() * q + (alpha/2)*q.dag()*q.dag()*q*q  + (k6/6)*(q.dag()*q.dag()*q.dag()*q*q*q) - g*(c.dag() - c)*(q.dag() - q)

H = np.array(H0)

cav_0 = np.zeros(Nq)
cav_1 = np.zeros(Nq)

for i in range(0,Nc*Nq):

    eigval = H0.eigenenergies()[i]
    eigvect = np.abs(np.reshape(np.array(H0.eigenstates()[1][i]),(Nc,Nq)))
    cav_is, qb_is = np.where(eigvect==np.max(eigvect))
    cav_i = cav_is[0]
    qb_i = qb_is[0]
    
    if cav_i == 0:
        cav_0[qb_i] = eigval
        
    elif cav_i == 1:
        cav_1[qb_i] = eigval
      

print('Chi from the model: ' + str(np.diff((cav_1-cav_0)/(2e6*np.pi))))
print('Dressed qb freq: ' + str((cav_0[1]-cav_0[0])/(2e9*np.pi)))
print('Dressed cav freq: ' + str((cav_1[0]-cav_0[0])/(2e9*np.pi)))

Chi from the model: [-0.33967082 -0.30247786 -0.26486633 -0.23061692 -3.64152136]
Dressed qb freq: 3.6209222888415318
Dressed cav freq: 5.729601864176592


In [47]:
H0.eigenstates()

(array([-2.37861541e+06,  2.27534231e+10,  3.60025277e+10,  5.87583294e+10]),
 array([Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
        Qobj data =
        [[0.99997976]
         [0.        ]
         [0.        ]
         [0.00636237]]                                                     ,
        Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
        Qobj data =
        [[ 0.        ]
         [-0.9996015 ]
         [ 0.02822821]
         [ 0.        ]]                                                    ,
        Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
        Qobj data =
        [[ 0.        ]
         [-0.02822821]
         [-0.9996015 ]
         [ 0.        ]]                                                    ,
        Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
        Qobj data =
        [[-0.00636237]
         [ 0.        ]
         [ 0.        ]
         [ 0.99997976]]           

In [11]:
np.diff(cav_0/(2e9*np.pi))

array([3.62092229, 3.3982202 , 3.1494701 , 2.87466932, 2.57381883,
       2.24692082, 1.89397688, 1.51500294, 1.11548518])

An attempt to fit the first-principles model to the most experimental data I can. This includes the dressed cavity frequency for four observed qubit states, and the dressed qubit transition frequency for the first and second transitions (equivalent to the anharmonicity).

From this we can get the fit bare cavity frequency.

The cavity frequencies are 5.72965, 5.72931, 5.72896 and 5.72867 GHz. The odd extra peaks are at (roughly) 5.7267 and 5.7305 GHz. 

The qubit frequencies are 3.620 GHz and 3.397 GHz. 

## Fit Constant-g Model

In [19]:
from scipy.optimize import minimize
import numpy as np
from qutip import *

def cost_function(x, *args):

    w_c_bare = x[0]
    w_q_bare = x[1]
    g0 = x[2]
    g1 = x[3]
    g2 = x[4]
    g3 = x[5]
    
    w_c_0_exp = args[0]
    chi_01_exp = args[1]
    chi_12_exp = args[2]
    chi_32_exp = args[3]
    w_q_0_exp = args[4]
    alpha = args[5]
    k6 = args[6]

    Nc = 6
    Nq = 6

    c  = tensor(destroy(Nc), qeye(Nq))

    q = tensor(qeye(Nc), destroy(Nq))

    tlist = np.linspace(0,1e-7,10000)

    q_0 = tensor(qeye(Nc), tensor(basis(Nq,0),basis(Nq,0).dag()))
    q_1 = tensor(qeye(Nc), tensor(basis(Nq,0),basis(Nq,0).dag()))
    q_2 = tensor(qeye(Nc), tensor(basis(Nq,0),basis(Nq,0).dag()))
    q_3 = tensor(qeye(Nc), tensor(basis(Nq,0),basis(Nq,0).dag()))

    H0 = w_c_bare * c.dag() * c + w_q_bare * q.dag() * q + (alpha/2)*q.dag()*q.dag()*q*q  + (k6/6)*(q.dag()*q.dag()*q.dag()*q*q*q)

    Hint = q*0
    
    gs = np.zeros(Nq) + g0
    gs[1] = g1
    gs[2] = g2
    gs[3] = g3

    for i in range(0, Nq):
    
        Hint += gs[i]*(c.dag() - c)*(q.dag() - q)*tensor(qeye(Nc), basis(Nq,i).proj())

    H = H0 + Hint

    cav_0 = np.zeros(Nq)
    cav_1 = np.zeros(Nq)

    for i in range(0,Nc*Nq):

        eigval = H.eigenenergies()[i]
        eigvect = np.abs(np.reshape(np.array(H.eigenstates()[1][i]),(Nc,Nq)))
        cav_is, qb_is = np.where(eigvect==np.max(eigvect))
        cav_i = cav_is[0]
        qb_i = qb_is[0]

        if cav_i == 0:
            cav_0[qb_i] = eigval

        elif cav_i == 1:
            cav_1[qb_i] = eigval

    w_cs = cav_1-cav_0

    w_c_0_fit = w_cs[0]
    w_c_1_fit = w_cs[1]
    w_c_2_fit = w_cs[2]
    w_c_3_fit = w_cs[3]

    chi_01_fit = w_c_1_fit-w_c_0_fit
    chi_12_fit = w_c_2_fit-w_c_1_fit
    chi_23_fit = w_c_3_fit-w_c_2_fit

    w_q_0_fit = cav_0[1]-cav_0[0]
    w_q_1_fit = cav_1[1]-cav_1[0]

    exp_values = np.array([w_c_0_exp, chi_01_exp, chi_12_exp, chi_23_exp, w_q_0_exp])
    fit_values = np.array([w_c_0_fit, chi_01_fit, chi_12_fit, chi_23_fit, w_q_0_fit])

#     exp_values = np.array([w_c_0_exp, w_q_0_exp])
#     fit_values = np.array([w_c_0_fit, w_q_0_fit])

    rel_error = np.abs( (exp_values-fit_values)/exp_values )

    cost = np.log(np.sum(rel_error))
    
    print(exp_values/(2*np.pi))
    print(fit_values/(2*np.pi))
    print(x/(2*np.pi))
    print(cost)
    print(' ')
    
    return cost

w_c_bare  = 2 * np.pi * 5.729e9 # resonator frequency
w_q_bare  = 2 * np.pi * 3.61e9 # qubit frequency
g0 = 60e6 * 2 * np.pi
g1 = 60e6 * 2 * np.pi
g2 = 60e6 * 2 * np.pi
g3 = 60e6 * 2 * np.pi

x0 = [w_c_bare, w_q_bare, g0, g1, g2, g3] # Guesses for the bare frequency


w_c_0_exp = 2 * np.pi * 5.72965e9 # resonator frequency
chi_01_exp = -2 * np.pi * 0.350e6
chi_12_exp = -2 * np.pi * 0.356e6
chi_23_exp = -2 * np.pi * 0.295e6
w_q_0_exp = 2 * np.pi * 3.62e9 # dressed qubit frequency
alpha = -2 * np.pi * 223e6 # resonator frequency
k6 = -26e6 * 2 * np.pi # resonator frequency

args = (w_c_0_exp, chi_01_exp, chi_12_exp, chi_23_exp, w_q_0_exp, alpha, k6)

res = minimize(cost_function, x0, args=args, method='Nelder-Mead', tol=1e-3, options={'maxiter': 30})

res

[ 5.72965e+09 -3.50000e+05 -3.56000e+05 -2.95000e+05  3.62000e+09]
[ 5.73031228e+09 -3.41439554e+05 -3.04682874e+05 -2.67328416e+05
  3.60789771e+09]
[5.729e+09 3.610e+09 6.000e+07]
-1.3247538725091472
 
[ 5.72965e+09 -3.50000e+05 -3.56000e+05 -2.95000e+05  3.62000e+09]
[ 6.01657183e+09 -2.71095797e+05 -2.50580515e+05 -2.27174224e+05
  3.60811224e+09]
[6.01545e+09 3.61000e+09 6.00000e+07]
-0.21711028939717222
 
[ 5.72965e+09 -3.50000e+05 -3.56000e+05 -2.95000e+05  3.62000e+09]
[ 5.73047738e+09 -4.00003232e+05 -3.46723907e+05 -2.95886576e+05
  3.78824796e+09]
[5.7290e+09 3.7905e+09 6.0000e+07]
-1.5207409090014894
 
[ 5.72965e+09 -3.50000e+05 -3.56000e+05 -2.95000e+05  3.62000e+09]
[ 5.73044666e+09 -3.76329663e+05 -3.35849894e+05 -2.94695497e+05
  3.60768233e+09]
[5.729e+09 3.610e+09 6.300e+07]
-1.9921418648065092
 
[ 5.72965e+09 -3.50000e+05 -3.56000e+05 -2.95000e+05  3.62000e+09]
[ 5.44437336e+09 -5.35864838e+05 -4.45919327e+05 -3.66957561e+05
  3.72765075e+09]
[5.44255000e+09 3.730333

[ 5.72965e+09 -3.50000e+05 -3.56000e+05 -2.95000e+05  3.62000e+09]
[ 5.73893555e+09 -3.75547368e+05 -3.35272097e+05 -2.94275178e+05
  3.61314021e+09]
[5.73748999e+09 3.61545602e+09 6.30206153e+07]
-1.9863918440264556
 
[ 5.72965e+09 -3.50000e+05 -3.56000e+05 -2.95000e+05  3.62000e+09]
[ 5.73319562e+09 -3.78477353e+05 -3.36652185e+05 -2.94455045e+05
  3.63206998e+09]
[5.73174658e+09 3.63437669e+09 6.26093224e+07]
-1.9553705033639448
 
[ 5.72965e+09 -3.50000e+05 -3.56000e+05 -2.95000e+05  3.62000e+09]
[ 5.74103031e+09 -3.74529206e+05 -3.35072797e+05 -2.94699344e+05
  3.60083481e+09]
[5.73958439e+09 3.60316010e+09 6.33147411e+07]
-1.9865534869058294
 
[ 5.72965e+09 -3.50000e+05 -3.56000e+05 -2.95000e+05  3.62000e+09]
[ 5.73929843e+09 -3.75417181e+05 -3.35772263e+05 -2.95236839e+05
  3.60114934e+09]
[5.73784987e+09 3.60347796e+09 6.33346718e+07]
-1.98679557699846
 
[ 5.72965e+09 -3.50000e+05 -3.56000e+05 -2.95000e+05  3.62000e+09]
[ 5.73604910e+09 -3.77042425e+05 -3.36446135e+05 -2.9517462

 final_simplex: (array([[3.60855546e+10, 2.27430098e+10, 3.96738417e+08],
       [3.60391875e+10, 2.27424622e+10, 3.95664573e+08],
       [3.60932892e+10, 2.27152283e+10, 3.97211599e+08],
       [3.60315488e+10, 2.27199088e+10, 3.96178917e+08]]), array([-2.00213425, -2.00007928, -1.99869119, -1.99768004]))
           fun: -2.0021342548799086
       message: 'Maximum number of iterations has been exceeded.'
          nfev: 50
           nit: 30
        status: 2
       success: False
             x: array([3.60855546e+10, 2.27430098e+10, 3.96738417e+08])

## Establish Variable-g Model

In [24]:
import numpy as np
from qutip import *

Nc = 10
Nq = 10

w_c  = 2 * np.pi * 5.729e9 # resonator frequency
w_q  = 2 * np.pi * 3.62e9 # qubit frequency
alpha = -223e6 * 2 * np.pi

gs = np.zeros(Nq) + 60e6 * 2 * np.pi
gs[0] = 60e6 * 2 * np.pi
gs[1] = 60e6 * 2 * np.pi
gs[2] = 60e6 * 2 * np.pi
gs[3] = 60e6 * 2 * np.pi

k6 = -26e6 * 2 * np.pi

c  = tensor(destroy(Nc), qeye(Nq))

q = tensor(qeye(Nc), destroy(Nq))

tlist = np.linspace(0,1e-7,10000)

interaction = q*0

for i in range(0, Nq):
    
    interaction += gs[i]*(c.dag() - c)*(q.dag() - q)*tensor(qeye(Nc), basis(Nq,i).proj())

H0 = w_c * c.dag() * c + w_q * q.dag() * q + (alpha/2)*q.dag()*q.dag()*q*q  + (k6/6)*(q.dag()*q.dag()*q.dag()*q*q*q) - interaction

H = np.array(H0)

cav_0 = np.zeros(Nq)
cav_1 = np.zeros(Nq)

for i in range(0,Nc*Nq):

    eigval = H0.eigenenergies()[i]
    eigvect = np.abs(np.reshape(np.array(H0.eigenstates()[1][i]),(Nc,Nq)))
    cav_is, qb_is = np.where(eigvect==np.max(eigvect))
    cav_i = cav_is[0]
    qb_i = qb_is[0]

    if cav_i == 0:
        cav_0[qb_i] = eigval

    elif cav_i == 1:
        cav_1[qb_i] = eigval

w_cs = cav_1-cav_0

print('Chi from the model: ' + str(np.diff((cav_1-cav_0)/(2e6*np.pi))))
print('Dressed qb freq: ' + str((cav_0[1]-cav_0[0])/(2e9*np.pi)))
print('Dressed cav freq: ' + str((cav_1[0]-cav_0[0])/(2e9*np.pi)))

Chi from the model: [-0.34430971 -0.30678279 -0.26878182 -0.23481682 -0.20731925 -0.18761198
 -0.17634233 -0.17687442 -1.69084896]
Dressed qb freq: 3.6178901277780837
Dressed cav freq: 5.730320728898739


In [39]:
from scipy.optimize import minimize
import numpy as np
from qutip import *

def cost_function(x, *args):

    w_c_bare = x[0]
    w_q_bare = x[1]
    g0 = x[2]
    g1 = x[3]
    g2 = x[4]
    g3 = x[5]
    
    w_c_0_exp = args[0]
    chi_01_exp = args[1]
    chi_12_exp = args[2]
    chi_32_exp = args[3]
    w_q_0_exp = args[4]
    alpha = args[5]
    k6 = args[6]

    Nc = 6
    Nq = 6

    c  = tensor(destroy(Nc), qeye(Nq))

    q = tensor(qeye(Nc), destroy(Nq))

    tlist = np.linspace(0,1e-7,10000)

    q_0 = tensor(qeye(Nc), tensor(basis(Nq,0),basis(Nq,0).dag()))
    q_1 = tensor(qeye(Nc), tensor(basis(Nq,0),basis(Nq,0).dag()))
    q_2 = tensor(qeye(Nc), tensor(basis(Nq,0),basis(Nq,0).dag()))
    q_3 = tensor(qeye(Nc), tensor(basis(Nq,0),basis(Nq,0).dag()))

    H0 = w_c_bare * c.dag() * c + w_q_bare * q.dag() * q + (alpha/2)*q.dag()*q.dag()*q*q  + (k6/6)*(q.dag()*q.dag()*q.dag()*q*q*q)

    Hint = q*0
    
    gs = np.zeros(Nq) + g0
    gs[1] = g1
    gs[2] = g2
    gs[3] = g3

    for i in range(0, Nq):
    
        Hint += gs[i]*(c.dag() - c)*(q.dag() - q)*tensor(qeye(Nc), basis(Nq,i).proj())

    H = H0 + Hint

    cav_0 = np.zeros(Nq)
    cav_1 = np.zeros(Nq)

    for i in range(0,Nc*Nq):

        eigval = H.eigenenergies()[i]
        eigvect = np.abs(np.reshape(np.array(H.eigenstates()[1][i]),(Nc,Nq)))
        cav_is, qb_is = np.where(eigvect==np.max(eigvect))
        cav_i = cav_is[0]
        qb_i = qb_is[0]

        if cav_i == 0:
            cav_0[qb_i] = eigval

        elif cav_i == 1:
            cav_1[qb_i] = eigval

    w_cs = cav_1-cav_0

    w_c_0_fit = w_cs[0]
    w_c_1_fit = w_cs[1]
    w_c_2_fit = w_cs[2]
    w_c_3_fit = w_cs[3]

    chi_01_fit = w_c_1_fit-w_c_0_fit
    chi_12_fit = w_c_2_fit-w_c_1_fit
    chi_23_fit = w_c_3_fit-w_c_2_fit

    w_q_0_fit = cav_0[1]-cav_0[0]
    w_q_1_fit = cav_1[1]-cav_1[0]

    exp_values = np.array([w_c_0_exp, chi_01_exp, chi_12_exp, chi_23_exp, w_q_0_exp])
    fit_values = np.array([w_c_0_fit, chi_01_fit, chi_12_fit, chi_23_fit, w_q_0_fit])

#     exp_values = np.array([w_c_0_exp, w_q_0_exp])
#     fit_values = np.array([w_c_0_fit, w_q_0_fit])

    fit_factor = [1,0.1,0.1,0.1,1]

    rel_error = np.abs( (exp_values-fit_values)/exp_values )

    cost = np.log(np.sum(rel_error*fit_factor))
    
    print(exp_values/(2*np.pi))
    print(fit_values/(2*np.pi))
    print(x/(2*np.pi))
    print(cost)
    print(rel_error)
    print(' ')
    
    return cost

w_c_bare  = 2 * np.pi * 5.7282e9 # initial guess bare resonator frequency
w_q_bare  = 2 * np.pi * 3.622e9 # guess bare qubit frequency
g0 = 60.7e6 * 2 * np.pi
g1 = 62.6e6 * 2 * np.pi
g2 = 61.1e6 * 2 * np.pi
g3 = 62.2e6 * 2 * np.pi

x0 = [w_c_bare, w_q_bare, g0, g1, g2, g3] # Guesses for the bare frequency


w_c_0_exp = 2 * np.pi * 5.72965e9 # resonator frequency
chi_01_exp = -2 * np.pi * 0.350e6
chi_12_exp = -2 * np.pi * 0.356e6
chi_23_exp = -2 * np.pi * 0.295e6
w_q_0_exp = 2 * np.pi * 3.62e9 # dressed qubit frequency
alpha = -2 * np.pi * 223e6 # resonator frequency
k6 = -26e6 * 2 * np.pi # resonator frequency

args = (w_c_0_exp, chi_01_exp, chi_12_exp, chi_23_exp, w_q_0_exp, alpha, k6)

res = minimize(cost_function, x0, args=args, method='Nelder-Mead', tol=1e-3, options={'maxiter': 100})

res

 C:\Users\boris\AppData\Local\Temp\ipykernel_1880\1867027788.py: 63ComplexWarning: Casting complex values to real discards the imaginary part
 C:\Users\boris\AppData\Local\Temp\ipykernel_1880\1867027788.py: 66

[ 5.72965e+09 -3.50000e+05 -3.56000e+05 -2.95000e+05  3.62000e+09]
[ 5.72959641e+09 -3.48250130e+05 -3.55808533e+05 -2.91894452e+05
  3.61976526e+09]
[5.7282e+09 3.6220e+09 6.0700e+07 6.2600e+07 6.1100e+07 6.2200e+07]
-6.3885607320296085
[9.35351609e-06 4.99962915e-03 5.37828409e-04 1.05272808e-02
 6.48460542e-05]
 
[ 5.72965e+09 -3.50000e+05 -3.56000e+05 -2.95000e+05  3.62000e+09]
[ 6.01580297e+09 -2.75011573e+05 -2.93683660e+05 -2.48369576e+05
  3.61999421e+09]
[6.01461e+09 3.62200e+09 6.07000e+07 6.26000e+07 6.11000e+07 6.22000e+07]
-2.2568389453916433
[4.99424869e-02 2.14252647e-01 1.75045900e-01 1.58069235e-01
 1.59911687e-06]
 
[ 5.72965e+09 -3.50000e+05 -3.56000e+05 -2.95000e+05  3.62000e+09]
[ 5.72977339e+09 -4.09513957e+05 -4.04767989e+05 -3.23279011e+05
  3.80070460e+09]
[5.7282e+09 3.8031e+09 6.0700e+07 6.2600e+07 6.1100e+07 6.2200e+07]
-2.40540551647164
[2.15348347e-05 1.70039877e-01 1.36988734e-01 9.58610550e-02
 4.99183975e-02]
 
[ 5.72965e+09 -3.50000e+05 -3.56000e+05 -2

[ 5.72965e+09 -3.50000e+05 -3.56000e+05 -2.95000e+05  3.62000e+09]
[ 5.75312055e+09 -3.55292399e+05 -3.87738741e+05 -2.74125164e+05
  3.64402267e+09]
[5.75169512e+09 3.64629757e+09 6.09204734e+07 6.35458800e+07
 6.13093282e+07 6.26332758e+07]
-3.567152777048586
[0.00409633 0.01512114 0.08915377 0.07076216 0.0066361 ]
 
[ 5.72965e+09 -3.50000e+05 -3.56000e+05 -2.95000e+05  3.62000e+09]
[ 5.77289385e+09 -3.54816721e+05 -4.40764161e+05 -2.27503926e+05
  3.66835900e+09]
[5.77146823e+09 3.67062673e+09 6.10788399e+07 6.31280688e+07
 6.14745378e+07 6.11751170e+07]
-2.6740458084476617
[0.00754738 0.01376206 0.23810158 0.22880025 0.01335884]
 
[ 5.72965e+09 -3.50000e+05 -3.56000e+05 -2.95000e+05  3.62000e+09]
[ 5.75488069e+09 -3.50735834e+05 -3.21869649e+05 -3.23822321e+05
  3.64219399e+09]
[5.75347247e+09 3.64444327e+09 6.09370678e+07 6.29002936e+07
 6.13251296e+07 6.32056896e+07]
-3.503158307325133
[0.00440353 0.00210238 0.09587177 0.09770278 0.00613094]
 
[ 5.72965e+09 -3.50000e+05 -3.56000e

[ 5.72965e+09 -3.50000e+05 -3.56000e+05 -2.95000e+05  3.62000e+09]
[ 5.72081091e+09 -3.55460009e+05 -3.45711157e+05 -2.89464671e+05
  3.60802175e+09]
[5.71941053e+09 3.61026469e+09 6.09369830e+07 6.26872501e+07
 6.11842057e+07 6.24249044e+07]
-4.493797853932073
[0.00154269 0.01560003 0.02890124 0.01876383 0.00330891]
 
[ 5.72965e+09 -3.50000e+05 -3.56000e+05 -2.95000e+05  3.62000e+09]
[ 5.73824104e+09 -3.47134610e+05 -3.56282009e+05 -2.99081075e+05
  3.62317245e+09]
[5.73683745e+09 3.62542073e+09 6.09564303e+07 6.28343409e+07
 6.14144145e+07 6.24847291e+07]
-5.369366347079228
[0.0014994  0.00818683 0.00079216 0.01383415 0.00087637]
 
[ 5.72965e+09 -3.50000e+05 -3.56000e+05 -2.95000e+05  3.62000e+09]
[ 5.71696516e+09 -3.43311987e+05 -3.52519262e+05 -3.07038258e+05
  3.60525952e+09]
[5.71556864e+09 3.60750044e+09 6.07671322e+07 6.26608825e+07
 6.12987551e+07 6.24079518e+07]
-4.323364580731853
[0.00221389 0.01910861 0.00977735 0.04080765 0.00407195]
 
[ 5.72965e+09 -3.50000e+05 -3.56000e+

[ 5.72965e+09 -3.50000e+05 -3.56000e+05 -2.95000e+05  3.62000e+09]
[ 5.72419477e+09 -3.51545153e+05 -3.56025301e+05 -2.95465996e+05
  3.62161243e+09]
[5.72278890e+09 3.62385892e+09 6.07606332e+07 6.26886901e+07
 6.11552592e+07 6.23336478e+07]
-6.212574052574044
[9.52104423e-04 4.41472210e-03 7.10700662e-05 1.57964787e-03
 4.45423805e-04]
 
[ 5.72965e+09 -3.50000e+05 -3.56000e+05 -2.95000e+05  3.62000e+09]
[ 5.72555985e+09 -3.50589060e+05 -3.59348613e+05 -2.91347854e+05
  3.62004352e+09]
[5.72415676e+09 3.62228709e+09 6.07210653e+07 6.27185772e+07
 6.11154352e+07 6.22817857e+07]
-5.785160111551781
[7.13857142e-04 1.68302784e-03 9.40621676e-03 1.23801543e-02
 1.20213143e-05]
 
[ 5.72965e+09 -3.50000e+05 -3.56000e+05 -2.95000e+05  3.62000e+09]
[ 5.72915352e+09 -3.50456489e+05 -3.54172773e+05 -2.96030488e+05
  3.61980514e+09]
[5.72774951e+09 3.62205175e+09 6.08629162e+07 6.27541523e+07
 6.12574354e+07 6.24206582e+07]
-6.7824537992791125
[8.66513008e-05 1.30425441e-03 5.13266112e-03 3.49317

[ 5.72965e+09 -3.50000e+05 -3.56000e+05 -2.95000e+05  3.62000e+09]
[ 5.72837710e+09 -3.50077266e+05 -3.55983802e+05 -2.95229142e+05
  3.61977524e+09]
[5.72697336e+09 3.62202127e+09 6.08218985e+07 6.27573313e+07
 6.12259147e+07 6.23927932e+07]
-7.853091227054934
[2.22160409e-04 2.20760481e-04 4.55008423e-05 7.76751587e-04
 6.20873166e-05]
 
[ 5.72965e+09 -3.50000e+05 -3.56000e+05 -2.95000e+05  3.62000e+09]
[ 5.72987569e+09 -3.49321886e+05 -3.56998661e+05 -2.94641712e+05
  3.62064407e+09]
[5.72847305e+09 3.62288851e+09 6.07938730e+07 6.27565841e+07
 6.12073690e+07 6.23707459e+07]
-7.114740229465343
[3.93890901e-05 1.93746843e-03 2.80522845e-03 1.21453588e-03
 1.77919520e-04]
 
[ 5.72965e+09 -3.50000e+05 -3.56000e+05 -2.95000e+05  3.62000e+09]
[ 5.72902347e+09 -3.50136251e+05 -3.55624309e+05 -2.95348780e+05
  3.62016346e+09]
[5.72761967e+09 3.62240956e+09 6.08329073e+07 6.27564224e+07
 6.12348974e+07 6.23976392e+07]
-7.781959254344309
[1.09348399e-04 3.89288841e-04 1.05531048e-03 1.182304

[ 5.72965e+09 -3.50000e+05 -3.56000e+05 -2.95000e+05  3.62000e+09]
[ 5.72934875e+09 -3.49830214e+05 -3.56193978e+05 -2.95258585e+05
  3.61965532e+09]
[5.72794506e+09 3.62190177e+09 6.08410068e+07 6.27757218e+07
 6.12482508e+07 6.24075053e+07]
-7.991142945607705
[5.25768946e-05 4.85103532e-04 5.44881080e-04 8.76558141e-04
 9.52158658e-05]
 
[ 5.72965e+09 -3.50000e+05 -3.56000e+05 -2.95000e+05  3.62000e+09]
[ 5.72956141e+09 -3.50039510e+05 -3.55906496e+05 -2.94934499e+05
  3.62008933e+09]
[5.72815784e+09 3.62233532e+09 6.08313702e+07 6.27703805e+07
 6.12327925e+07 6.24039445e+07]
-9.211372965115311
[1.54611815e-05 1.12886480e-04 2.62651377e-04 2.22038833e-04
 2.46779435e-05]
 
[ 5.72965e+09 -3.50000e+05 -3.56000e+05 -2.95000e+05  3.62000e+09]
[ 5.73017296e+09 -3.49903753e+05 -3.55965852e+05 -2.94933413e+05
  3.62001173e+09]
[5.72876947e+09 3.62225791e+09 6.08423396e+07 6.27813121e+07
 6.12450381e+07 6.24133820e+07]
-8.777418529403562
[9.12731440e-05 2.74990795e-04 9.59199788e-05 2.257172

[ 5.72965e+09 -3.50000e+05 -3.56000e+05 -2.95000e+05  3.62000e+09]
[ 5.72972033e+09 -3.50012756e+05 -3.55947569e+05 -2.94839515e+05
  3.62003419e+09]
[5.72831683e+09 3.62228016e+09 6.08347572e+07 6.27718487e+07
 6.12359265e+07 6.24039294e+07]
-9.266981667388675
[1.22751966e-05 3.64464135e-05 1.47277987e-04 5.44015920e-04
 9.44406731e-06]
 
[ 5.72965e+09 -3.50000e+05 -3.56000e+05 -2.95000e+05  3.62000e+09]
[ 5.72961052e+09 -3.50005912e+05 -3.55998091e+05 -2.95043851e+05
  3.62000703e+09]
[5.72820681e+09 3.62225334e+09 6.08372337e+07 6.27758813e+07
 6.12401510e+07 6.24090779e+07]
-10.560425810256152
[6.88987240e-06 1.68914995e-05 5.36281357e-06 1.48647687e-04
 1.94173882e-06]
 
[ 5.72965e+09 -3.50000e+05 -3.56000e+05 -2.95000e+05  3.62000e+09]
[ 5.72967135e+09 -3.50027427e+05 -3.56025465e+05 -2.95016028e+05
  3.61996217e+09]
[5.72826757e+09 3.62220864e+09 6.08410616e+07 6.27789832e+07
 6.12436463e+07 6.24116879e+07]
-10.271679023210474
[3.72638336e-06 7.83618699e-05 7.15301088e-05 5.4333

[ 5.72965e+09 -3.50000e+05 -3.56000e+05 -2.95000e+05  3.62000e+09]
[ 5.72965937e+09 -3.50014003e+05 -3.55998110e+05 -2.95031504e+05
  3.61999459e+09]
[5.72825564e+09 3.62224096e+09 6.08397348e+07 6.27765180e+07
 6.12424459e+07 6.24096415e+07]
-10.906367006845414
[1.63582716e-06 4.00071737e-05 5.30987065e-06 1.06793292e-04
 1.49422281e-06]
 
[ 5.72965e+09 -3.50000e+05 -3.56000e+05 -2.95000e+05  3.62000e+09]
[ 5.72964421e+09 -3.50000499e+05 -3.56007671e+05 -2.95005293e+05
  3.61996643e+09]
[5.72824052e+09 3.62221273e+09 6.08387277e+07 6.27759653e+07
 6.12414545e+07 6.24086760e+07]
-11.149995035898943
[1.01110712e-06 1.42469012e-06 2.15486093e-05 1.79419355e-05
 9.27272783e-06]
 
[ 5.72965e+09 -3.50000e+05 -3.56000e+05 -2.95000e+05  3.62000e+09]
[ 5.72964387e+09 -3.49998425e+05 -3.56001014e+05 -2.94993734e+05
  3.62001513e+09]
[5.72824022e+09 3.62226135e+09 6.08368009e+07 6.27746494e+07
 6.12394721e+07 6.24073406e+07]
-11.722787827100793
[1.06906632e-06 4.50066334e-06 2.84787536e-06 2.124

       message: Maximum number of iterations has been exceeded.
       success: False
        status: 2
           fun: -12.423316407239394
             x: [ 3.599e+10  2.276e+10  3.823e+08  3.944e+08  3.848e+08
                  3.921e+08]
           nit: 100
          nfev: 188
 final_simplex: (array([[ 3.599e+10,  2.276e+10, ...,  3.848e+08,
                         3.921e+08],
                       [ 3.599e+10,  2.276e+10, ...,  3.848e+08,
                         3.921e+08],
                       ...,
                       [ 3.599e+10,  2.276e+10, ...,  3.848e+08,
                         3.921e+08],
                       [ 3.599e+10,  2.276e+10, ...,  3.848e+08,
                         3.921e+08]]), array([-1.242e+01, -1.211e+01, -1.209e+01, -1.199e+01,
                       -1.195e+01, -1.185e+01, -1.174e+01]))